In [ ]:
from additional.nearest_larger_item import solution_minheap, solution_stack
a = [3, 2, 1, 4]
r1 = solution_minheap(a)
r2 = solution_stack(a)
assert r1 == r2


In [ ]:
a = [1, 2, 3]
a.pop()


In [ ]:
from additional.robot_up_right import solution_bottom_up, solution_top_down
assert solution_bottom_up(5, 4) == solution_top_down(5, 4)


In [ ]:
from lesson3.n2_hash_chains import Hash
h = Hash(5)
h('Hell0'), h('world'), h('luck'), h('GooD'), h('Hell0'), h('GooD')


In [ ]:
h.powers


In [ ]:
[ord(c) for c in 'Hell0']


In [ ]:
p = 1000000007
x = 263
powers = [263**i % 1000000007 for i in range(5)]
(72 + (111*powers[1]) + (114 * powers[2]) +
 (108 * powers[3]) + 100 * powers[4]) % 5


In [ ]:
sum([(ord(ch)*h.powers[i]) for i, ch in enumerate('hell0')]) % 5


In [ ]:
from lesson3.n3_pattern_in_text import solve, Hash

assert solve('aba', 'aba') == [0]
assert solve('abacaba', 'aba') == [0, 4]
assert solve('ababacaba', 'aba') == [0, 2, 6]
assert solve('testTesttesT', 'Test') == [4]
assert solve('baaaaaaa', 'aaaaa') == [1, 2, 3]

h = Hash(x=500)
s = 'hello'
old_hash = h(s)
ns = 'ellow'
assert h.get_new_hash(len(s), s[0], ns[-1], old_hash) == h(ns)


In [ ]:
from lesson4.n4_tree import Tree, Node, merge_trees, split_tree, solve
from typing import List, Tuple, Union, Callable

NodeT = Tuple[int, Union[int, None]]
InOrderResult = List[NodeT]

def in_order(node: Node):
    if node is None:
        return
    for n in in_order(node.l):
        yield n
    yield node
    for n in in_order(node.r):
        yield n

def are_equal(expected, actual):
    assert expected == actual, f"Expected: {expected}, Actual: {actual}"

def check_tree(t: Tree):
    def h(node: Node):
        return 0 if node is None else max(node.lh, node.rh) + 1
    def s(node: Node):
        return 0 if node is None else node.ls + node.rs + node.k
    
    def check_node(node: Node):
        if node is None:
            return
        try: 
            are_equal(h(node.l), node.lh)
            are_equal(h(node.r), node.rh)
            # are_equal(s(node.l), node.ls)
            # are_equal(s(node.r), node.rs)
            assert abs(node.lh - node.rh) <= 1, f"lh: {node.lh} rh: {node.rh}"
            check_node(node.l)
            check_node(node.r)
        except AssertionError:
            raise

    check_node(t.root)

    
def show_tree(tree: Tree):
    for n in in_order(tree.root):
        h = max(n.lh, n.rh)
        print(''.join(['\t']*h) + repr(n))


def test_tree(t: Tree, result: InOrderResult):
    check_tree(t)
    transform: Callable[[Node], NodeT] = lambda n: (
        n.k, n.p.k if n.p else None)
    actual = list(map(transform, in_order(t.root)))
    are_equal(result, actual)


def build_tree(nodes):
    t = Tree()
    for i in nodes:
        t.add(i)
    return t


def build_n_tree(n):
    return build_tree(list(range(n)))


In [ ]:
def test_tree_add():
    def test(nodes, result: InOrderResult):
        test_tree(build_tree(nodes), result)

    test([1], [(1, None)])
    test([1, 2], [(1, None), (2, 1)])
    test([2, 1], [(1, 2), (2, None)])
    # right small rotation
    test([1, 2, 3], [(1, 2), (2, None), (3, 2)])
    # left small rotation
    test([3, 2, 1], [(1, 2), (2, None), (3, 2)])
    # left big rotation
    test([4, 5, 1, 0, 2, 3], [(0, 1), (1, 2), (2, None), (3, 4), (4, 2), (5, 4)])
    # right big rotation
    test([1, 0, 4, 5, 3, 2], [(0, 1), (1, 3), (2, 1), (3, None), (4, 3), (5, 4)])


def test_tree_remove():
    def test(nodes: List[int], remove_node: int, result):
        t = build_tree(nodes)
        t.remove(remove_node)
        test_tree(t, result)

    test([0], 0, [])
    test([0, 1], 0, [(1, None)])
    test([0, 1, 2], 1, [(0, None), (2, 0)])

    nodes = list(range(5))
    test(nodes, 0, [(1, 3), (2, 1), (3, None), (4, 3)])
    test(nodes, 1, [(0, 3), (2, 0), (3, None), (4, 3)])
    test(nodes, 2, [(0, 1), (1, None), (3, 1), (4, 3)])
    test(nodes, 3, [(0, 1), (1, None), (2, 1), (4, 2)])
    test(nodes, 4, [(0, 1), (1, None), (2, 3), (3, 1)])


def test_tree_merge():
    def test(nodes1: Tree, nodes2: Tree, result: InOrderResult):
        t1 = build_tree(nodes1)
        t2 = build_tree(nodes2)
        t = merge_trees(t1, t2)
        test_tree(t, result)

    test([1], [2], [(1, 2), (2, None)])
    test([1], [2, 3], [(1, 2), (2, None), (3, 2)])
    test([1, 2], [3, 4], [(1, 3), (2, 1), (3, None), (4, 3)])
    test([1], [4, 5, 3, 2], [(1, 2), (2, 4), (3, 2), (4, None), (5, 4)])


def test_tree_split():
    def test(nodes: Tree, k, r1: InOrderResult, r2: InOrderResult):
        t = build_tree(nodes)
        t1, t2 = split_tree(t, k)
        test_tree(t1, r1)
        test_tree(t2, r2)

    test([1], 0, [], [(1, None)])
    test([1], 1, [(1, None)], [])
    test([1], 2, [(1, None)], [])
    test([1,2], 0, [], [(1, 2), (2, None)])
    test([1,2], 3, [(1, 2), (2, None)], [])
    test([1,2], 1, [(1, None)],[(2, None)])
    test([1,3,4], 2, [(1, None)],[(3, 4), (4, None)])

def test_solution():
    def test(commands, expected):
        actual = list(solve(map(lambda x: x.split(), commands)))
        are_equal(expected, actual)

    test(['+ 1'], [])
    test(['- 1'], [])
    test(['? 1'], ['Not found'])
    test(['+ 1','? 1'], ['Found'])
    test(['+ 1','- 1','? 1'], ['Not found'])
    test(['s 1 2'], ['0'])
    test(['+ 1','s 1 2'], ['1'])
    test(['+ 1','+ 2','s 1 2'], ['3'])
    test(['+ 1','+ 2','s 1 2','+ 1','s 1 2'], ['3','4'])
    test(['+ 1','+ 2','s 1 2','+ 1','? 1'], ['3','Found'])

    test(['+ 1','+ 2','s 1 2','+ 1','? 1'], ['3','Found'])

    test(['+ 1_000_000_001','? 0'], ['Found'])
    test(['+ 1_000_000_002','? 1'], ['Found'])
    test(['+ 1_000_000_000','s 0 1_000_000_000'], ['1000000000'])
    test(['+ 1_000_000_000','s 0 1_000_000_000', '? 0'], ['1000000000', 'Found'])
    test(['+ 1','+ 1_000_000_000','s 0 1_000_000_000', '? 1'], ['1000000001', 'Found'])

def random_test_tree():
    import random
    for i in range(100):
        nodes = list(range(100))
        random.shuffle(nodes)
        original = nodes.copy()
        t = build_tree(original)
        check_tree(t)
        random.shuffle(nodes)
        try:
            for k in nodes:
                t.remove(k)
                check_tree(t)
        except AssertionError as e:
            print(e, k)
            print(original)
            print(nodes)
            raise
        assert t.root is None

test_tree_add()
test_tree_remove()
test_tree_merge()
test_solution()
# random_test_tree()
test_tree_split()

In [ ]:
def perf_build():
    arr = list(range(10000))
    from time import time
    import random
    random.seed(0)
    random.shuffle(arr)
    beg = time()
    t = build_tree(arr)
    end = time()
    return end - beg

In [ ]:
import cProfile
cProfile.run('perf_build()')